In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score
from sklearn.model_selection import KFold
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

drive = 'gdrive/My Drive/TUBES_RECSYS/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **Collaborative Filtering using Matrix Factorization**

In [ ]:
# get ratings dataset
df_rating = pd.read_csv(drive + 'ratings.csv', sep=',')

# get movies dataset
df_movies = pd.read_csv(drive + 'movies.csv', sep=',')

In [ ]:
# display first 5 rows of rating dataset
df_rating.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
# display first 5 rows of movie dataset
df_movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
dataset_rating = df_rating
dataset_movies = df_movies

In [ ]:
# locate features and label for rating prediction
X = df_rating.iloc[:,[0,1]].values
y = df_rating.iloc[:,2].values

In [ ]:
"""
Input:
    U     : a set of users
    D     : a set of items
    R     : a matrix of dimension |U| x |D| that contains all users ratings towards a movie
    P     : an initial matrix of association between users and features with dimension |U| x K
    Q     : an initial matrix of association between items and features with dimension |D| x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimization
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""

def matrix_factorization(R, P, Q, K, steps=6, alpha=0.0002, beta=0.02):
    Q = Q.T # transpose matrix Q
    
    for step in range(steps):
      for i in range(len(R)):
          for j in range(len(R[i])):
              if R[i][j] > 0: # only minimise errors of the observed user-item pairs
                  eij = R[i][j] - np.dot(P[i,:],Q[:,j]) # count error between real rating and estimated rating
                  for k in range(K): 
                      P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k]) # set update rules for P
                      Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j]) # set update rules for Q
      eR = np.dot(P,Q)
      e = 0
      for i in range(len(R)):
          for j in range(len(R[i])):
              if R[i][j] > 0:
                  e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2) # check the overall error 
                  for k in range(K):
                      e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) ) # add beta to avoid overfitting
      if e < 0.001:
          break
    return P, Q.T

In [ ]:
cv = KFold(n_splits=5, random_state=42, shuffle=True) # do cross-validation for 5 folds

df_user_items_pred = [] # initialize array that stores all dataframe of user and items prediction
score = [] # initialize NDCG score of test set

for train_index, test_index in cv.split(X):
    
    df_rating = dataset_rating
    
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index] # split into data train and data test
    real_val = y_test # save real rating of data test
    
    for i in range(len(test_index)):
        df_rating['rating'].iloc[test_index[i]] = 0 # mask data test result
    
    df_rating = df_rating.pivot(index='userId', columns='movieId', values='rating') # change dataframe to matrix of user and item (movie)
    df_rating = df_rating.fillna(0) # fill unrated movie's ratings with 0
    
    R = df_rating.to_numpy() 

    U = len(R) # intialize user latent features matrix
    D = len(R[0]) # initialize item latent features matrix
    K = 4 # set number of latent features

    P = np.random.rand(U,K) # initialize user latent features matrix with random values
    Q = np.random.rand(D,K) # initialize item latent features matrix with random values

    print("The Original Matrix")
    print(R)

    user_latent_features , item_latent_features  = matrix_factorization(R, P, Q, K)

    user_items_matrix = np.dot(user_latent_features, item_latent_features.T)
    print("\nThe Approximation matrix by MF")
    print(user_items_matrix)
    
    df_user_items =  pd.DataFrame(user_items_matrix, columns = df_rating.columns, index = df_rating.index) # set user items matrix as dataframe
    df_user_items_pred.append(df_user_items)
    
    predicted = [] # initialize empty list for predicted data test
    for i in range(len(X_test)):
        predicted.append(df_user_items.loc[X_test[i][0]][X_test[i][1]]) # get rating prediction of data test
    
    predicted = np.asarray([predicted]) 
    real_val = np.asarray([real_val])
    score.append(ndcg_score(real_val,predicted))  # evaluate NDCG score of data test prediction

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


The Original Matrix
[[4.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 2.  2.  ... 0.  0.  0. ]
 [3.  0.  0.  ... 0.  0.  0. ]
 [5.  0.  0.  ... 0.  0.  0. ]]

The Approximation matrix by MF
[[3.93355118 3.51816958 2.90023802 ... 0.99344018 2.43608675 3.17497707]
 [2.8828714  2.44792476 2.00889575 ... 0.78666753 1.87176033 2.31429061]
 [1.99544556 1.55540968 1.47072015 ... 0.62817827 1.41822956 1.5639089 ]
 ...
 [4.52438748 3.68143995 2.82638576 ... 1.08180049 2.80391971 3.46119549]
 [2.95641747 2.33261766 1.9543789  ... 0.79679814 1.94682175 2.26764897]
 [5.31652507 4.61448665 3.81508727 ... 1.42735118 3.40872205 4.29181511]]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


The Original Matrix
[[4.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 0.  0.  ... 0.  0.  0. ]
 [3.  0.  0.  ... 0.  0.  0. ]
 [5.  0.  0.  ... 0.  0.  0. ]]

The Approximation matrix by MF
[[2.90838362 3.38804026 2.79416413 ... 1.43767089 1.51395786 2.23478706]
 [2.07891206 2.47430811 2.32896863 ... 0.89271945 1.3505083  1.31925064]
 [1.97129672 2.26062258 1.74708625 ... 1.09909826 0.87998249 1.65577926]
 ...
 [3.26986107 3.81796879 3.27127419 ... 1.61038079 1.78262261 2.42874411]
 [1.27565305 1.52113159 1.1067211  ... 0.61183842 0.66860017 0.87628543]
 [4.17598492 4.93719272 3.87484682 ... 2.21749363 2.22133854 2.84023249]]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


The Original Matrix
[[0.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 0.  0.  ... 0.  0.  0. ]
 [3.  0.  0.  ... 0.  0.  0. ]
 [5.  0.  0.  ... 0.  0.  0. ]]

The Approximation matrix by MF
[[2.7505596  1.73402065 1.53518037 ... 0.83624608 1.4744098  1.93863167]
 [1.63557902 0.98530107 1.26226059 ... 0.49310456 0.93334164 1.3602988 ]
 [1.90715374 1.14321729 1.05758142 ... 0.3089623  0.9267034  1.17832693]
 ...
 [3.50640733 2.72647922 1.82755835 ... 0.69916372 1.77648114 2.10402361]
 [2.03859539 1.53904955 0.79607634 ... 0.21946564 0.9202223  0.95616055]
 [4.63295717 3.40009618 2.4294215  ... 1.02640546 2.36763563 2.87719813]]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


The Original Matrix
[[0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]

The Approximation matrix by MF
[[1.8730311  2.2547664  2.29338119 ... 2.03164997 1.81140997 1.10087956]
 [1.08763028 1.42911902 1.55648582 ... 1.3507249  0.94646333 0.81189635]
 [1.58130366 2.15272412 2.27605707 ... 1.68992087 1.36204983 1.09807971]
 ...
 [1.37114427 1.88655819 2.11427157 ... 1.44018665 1.10662381 0.98602345]
 [1.48712266 1.38643424 1.40287356 ... 1.3705948  1.62732467 0.51904526]
 [2.41129195 2.71912426 2.96758433 ... 2.62117497 2.31409005 1.35757232]]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


The Original Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

The Approximation matrix by MF
[[0.40925971 0.22548503 0.0591031  ... 0.32943765 0.28144137 0.36759473]
 [1.16879957 1.01886582 0.33068396 ... 1.00549073 0.5774399  1.03520879]
 [1.21151993 0.88590136 0.30225539 ... 1.14687976 0.71341121 1.09085563]
 ...
 [1.21797234 0.84165576 0.24575409 ... 1.00218044 0.73160453 1.08348572]
 [2.13619523 1.4562597  0.38787921 ... 1.8605431  1.23245908 1.80876668]
 [1.40142735 1.05406037 0.2968452  ... 1.25558198 0.74502923 1.17650541]]


In [ ]:
# print NDCG score for 5 fold validation and its average
print("NDCG score : ", score)
print("NDCG average score : ", np.mean(score))

NDCG score :  [0.9759309390384933, 0.9746077361206373, 0.9722639105289763, 0.9681632445226911, 0.9687211633470916]
NDCG average score :  0.9719373987115778


In [ ]:
# get user items dataframe of best data prediction
df_user_items = df_user_items_pred[score.index(max(score))]
df_user_items

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.933551,3.518170,2.900238,2.191680,2.228772,3.470723,2.333630,2.337178,1.617999,3.142980,2.406717,2.477636,2.460978,2.548688,3.105085,2.467899,2.570575,3.089799,2.466033,2.535024,2.307281,1.165386,2.080718,0.943954,2.480314,2.236066,2.994162,2.873483,3.095408,3.185012,2.437456,3.933151,3.853861,2.922037,1.336782,3.668945,1.885620,1.895250,3.089494,2.980357,...,2.978562,2.604165,3.287825,3.908263,2.689202,3.025906,0.863219,2.560115,3.422883,1.974451,2.982466,2.897366,2.455920,2.593788,1.474776,2.592755,2.898438,2.536438,2.195493,1.919989,2.900504,2.233070,2.305862,1.155227,1.810754,2.448407,2.565241,1.443273,1.582396,3.803327,2.005426,2.177204,1.135564,3.561162,1.789010,1.766322,2.464406,0.993440,2.436087,3.174977
2,2.882871,2.447925,2.008896,1.778990,1.603670,2.300799,1.884748,1.657102,1.384422,2.311077,1.741864,1.463183,1.989687,1.889664,2.129608,1.957739,1.920177,2.245370,1.889040,1.927269,1.630053,0.840276,1.256013,0.721936,1.723581,1.597551,2.255585,2.010114,2.307036,2.146629,1.916212,2.638147,2.859429,1.914083,1.246788,2.418036,1.112245,1.447436,2.045210,2.234516,...,1.982895,1.930734,2.029215,2.680199,1.619614,2.266757,0.513410,1.890117,2.298314,1.692541,1.993815,1.809879,1.515647,1.885155,1.191997,1.835773,1.762272,1.830727,1.615304,1.298558,1.954700,1.638727,1.805653,0.921202,1.171579,1.783063,1.600985,1.079230,1.449067,2.350366,1.399998,1.629004,0.692003,2.230337,1.512485,1.113038,1.717107,0.786668,1.871760,2.314291
3,1.995446,1.555410,1.470720,1.384182,0.986945,1.619393,1.375066,1.451073,0.936067,1.794792,1.216156,1.114412,1.354316,1.246012,1.593197,1.457753,1.397241,1.609103,1.417771,1.466731,1.064829,0.550409,1.048366,0.529818,1.188063,1.163201,1.555462,1.605823,1.574980,1.693207,1.362196,1.768911,2.054971,1.530360,0.908029,1.588283,0.867067,0.988258,1.397656,1.619113,...,1.442255,1.474060,1.397481,1.926123,1.220123,1.619866,0.339587,1.415628,1.719965,1.156887,1.206980,1.302992,1.131999,1.288683,0.928837,1.230878,1.245729,1.168409,1.134569,0.845095,1.484245,1.229422,1.305383,0.670489,0.835004,1.435655,1.189770,0.757963,1.009357,1.612745,0.946243,1.232142,0.447367,1.594459,1.096219,0.712182,1.113697,0.628178,1.418230,1.563909
4,3.078355,2.690658,2.414947,2.054772,1.709250,2.800217,1.956482,2.087774,1.411768,2.697569,1.862694,1.854733,2.059951,2.044486,2.622071,2.101892,2.227340,2.520136,2.112931,2.272481,1.814562,0.906101,1.735443,0.824340,1.962710,1.856597,2.471339,2.445203,2.492691,2.588481,2.065697,3.043222,3.252302,2.410369,1.268919,2.820742,1.459050,1.485624,2.402111,2.453234,...,2.282755,2.180857,2.455183,3.127601,1.997876,2.529579,0.591465,2.199910,2.808203,1.792869,2.211296,2.170997,1.891077,1.992157,1.272345,2.029883,2.311307,2.019031,1.850395,1.388187,2.312260,1.949260,1.870638,1.042776,1.404755,2.151465,2.005901,1.226004,1.477096,2.880005,1.620635,1.830934,0.813023,2.735257,1.514156,1.291586,1.931838,0.918664,2.073177,2.533924
5,2.584809,2.447476,1.689851,1.418293,1.671801,2.032540,1.601456,0.967580,1.345874,1.767518,1.532304,1.065330,1.885322,1.838166,1.736135,1.616285,1.665317,1.939830,1.542011,1.569660,1.590693,0.813616,0.797184,0.619256,1.560474,1.354972,2.083310,1.440213,2.139738,1.536935,1.708402,2.460073,2.499757,1.341251,1.117782,2.313018,0.787157,1.327885,1.857451,1.903876,...,1.613493,1.507490,1.778457,2.280127,1.202503,1.991214,0.457353,1.551654,1.842166,1.646155,2.084863,1.476333,1.171895,1.708370,0.906959,1.724420,1.532698,1.852027,1.464180,1.226849,1.498783,1.352418,1.535096,0.813391,0.992678,1.279811,1.262

In [ ]:
# Get top-20 recommendations for all user
def recommendation(df_user_items, dataset_rating, dataset_movies):
    n_recommendations = 20 # set number of recommendations
    n_users = dataset_rating.userId.unique().shape[0] # get all unique users id count

    for i in range(1,n_users+1):
      user_rating_pred = df_user_items.loc[i].sort_values(ascending=False) # get user rating prediction
      user_rated = (dataset_rating[dataset_rating['userId'] == i].merge(dataset_movies, how = 'left', on='movieId').
                      sort_values(['rating'], ascending=False)) # get user rated movie information

      recommendations = (dataset_movies[~dataset_movies['movieId'].isin(user_rated['movieId'])].
            merge(pd.DataFrame(user_rating_pred).reset_index(), how = 'left',
                  on = 'movieId').
            rename(columns = {i: 'Predictions'}).
            sort_values('Predictions', ascending = False).
                          iloc[:n_recommendations, :-1]
                        ) # get 20-top rated movie recommendation for user that hasn't being watched by user

      print("User id = ", i)
      print("Recommended movies: ")
      for j in range(recommendations.shape[0]):
        print(j+1,".", recommendations['title'].values[j])
      print("\n")

recommendation(df_user_items, dataset_rating, dataset_movies)

Streaming output truncated to the last 5000 lines.
16 . One Flew Over the Cuckoo's Nest (1975)
17 . Death Note: Desu nôto (2006–2007)
18 . Memento (2000)
19 . Game, The (1997)
20 . Amadeus (1984)


User id =  403
Recommended movies: 
1 . Forrest Gump (1994)
2 . Shawshank Redemption, The (1994)
3 . Godfather, The (1972)
4 . Star Wars: Episode IV - A New Hope (1977)
5 . Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
6 . Star Wars: Episode VI - Return of the Jedi (1983)
7 . Fight Club (1999)
8 . Terminator 2: Judgment Day (1991)
9 . Schindler's List (1993)
10 . One Flew Over the Cuckoo's Nest (1975)
11 . Fugitive, The (1993)
12 . Matrix, The (1999)
13 . Star Wars: Episode V - The Empire Strikes Back (1980)
14 . Memento (2000)
15 . Amadeus (1984)
16 . Silence of the Lambs, The (1991)
17 . Dark Knight, The (2008)
18 . Usual Suspects, The (1995)
19 . Inception (2010)
20 . Game, The (1997)


User id =  404
Recommended movies: 
1 . Fight Club (1999)
2 . Goodfell

# **Content Based Filtering (clustering over Movie genre)**

In [ ]:
# get movie dataset
df_movies = dataset_movies
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
# display all unique genres in movie dataset
movie_genres = df_movies['genres'].str.split("|").to_list()
flatten_list = list(chain.from_iterable(movie_genres)) 
set(flatten_list)

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [ ]:
# delete movie data where genre is not listed
df_movies = df_movies[df_movies['genres'] != "(no genres listed)"]
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
# function to count cosine similarity of a specific column towards movie_id

def count_cosine_similarity(column, movies_id):
  # weight keyword tf-idf of column in dataset movies
  tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=0, token_pattern=r"(?u)\w[\w-]*\w")
  tfidf_matrix = tf.fit_transform(column)

  # calculate cosine similarities to find the column values similarity
  cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

  # get dataframe of movie id to column cosine similarities
  recommender_df = pd.DataFrame(cosine_similarities, columns=column, index=movies_id)

  # delete duplicate column (duplicated genres) in dataframe
  recommender_df = recommender_df.loc[:,~recommender_df.columns.duplicated()]
  return recommender_df

recommender_df = count_cosine_similarity(df_movies.genres, df_movies.movieId)
recommender_df

genres,Adventure|Animation|Children|Comedy|Fantasy,Adventure|Children|Fantasy,Comedy|Romance,Comedy|Drama|Romance,Comedy,Action|Crime|Thriller,Adventure|Children,Action,Action|Adventure|Thriller,Comedy|Horror,Adventure|Animation|Children,Drama,Action|Adventure|Romance,Crime|Drama,Drama|Romance,Action|Comedy|Crime|Drama|Thriller,Comedy|Crime|Thriller,Crime|Drama|Horror|Mystery|Thriller,Drama|Sci-Fi,Children|Drama,Adventure|Drama|Fantasy|Mystery|Sci-Fi,Mystery|Sci-Fi|Thriller,Children|Comedy,Drama|War,Action|Crime|Drama,Action|Adventure|Fantasy,Comedy|Drama|Thriller,Mystery|Thriller,Animation|Children|Drama|Musical|Romance,Crime|Mystery|Thriller,Adventure|Drama,Drama|Thriller,Comedy|Crime,Action|Sci-Fi|Thriller,Action|Comedy|Horror|Thriller,Comedy|Drama,Documentary,Action|Crime|Drama|Thriller,Crime|Drama|Romance,Action|Adventure|Drama,...,Adventure|Romance|Sci-Fi,Horror|Sci-Fi|Western,Action|Adventure|Children|Comedy|Sci-Fi,Action|Adventure|Animation|Comedy|Sci-Fi,Horror|Romance|Sci-Fi,Action|Adventure|Children|Mystery|Sci-Fi,Comedy|Crime|Sci-Fi,Action|Comedy|Fantasy|Sci-Fi,Adventure|Animation|Children|Comedy|Drama|Fantasy,Adventure|Children|Comedy|Sci-Fi,Action|Animation|Crime|Sci-Fi,Action|Adventure|Romance|Sci-Fi,Action|Crime|Drama|Sci-Fi,Animation|Children|Comedy|Drama|Romance,Action|Fantasy|Mystery,Comedy|Horror|Mystery,Comedy|Crime|Fantasy,Animation|Horror|Mystery,Comedy|Romance|Western,Adventure|Drama|Fantasy|Sci-Fi,Adventure|Children|Drama|Sci-Fi,Adventure|Children|Comedy|Drama|Fantasy|Sci-Fi,Animation|Fantasy|Horror|Mystery,Adventure|Animation|Fantasy|Horror|Sci-Fi,Action|Animation|Crime|Drama,Action|Adventure|Animation|Drama|Fantasy|Sci-Fi,Action|Animation|Mystery,Animation|Drama|Sci-Fi,Animation|Drama|Fantasy|Romance,Action|Adventure|Comedy|Fantasy|Sci-Fi|Thriller,Action|Adventure|Fantasy|Horror|Thriller,Comedy|Sci-Fi|War,Comedy|Mystery|Romance|Thriller,Fantasy|Horror|Sci-Fi|Western,Animation|Crime|Drama,Adventure|Mystery|Sci-Fi|Thriller,Action|Comedy|Crime|Horror,Action|Adventure|Children|Sci-Fi,Action|Adventure|Comedy|Fantasy|Sci-Fi,Action|Animation|Comedy|Fantasy
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.813604,0.152600,0.135000,0.267388,0.000000,0.654710,0.000000,0.262413,0.136174,0.833785,0.000000,0.257325,0.000000,0.000000,0.094145,0.115720,0.000000,0.000000,0.453566,0.418129,0.0,0.571328,0.000000,0.000000,0.553347,0.139422,0.0,0.499667,0.0,0.358605,0.000000,0.142647,0.000000,0.097274,0.196445,0.0,0.000000,0.000000,0.286092,...,0.239614,0.0,0.546316,0.555184,0.000000,0.419854,0.105978,0.380183,0.970845,0.596059,0.300996,0.213889,0.000000,0.662153,0.290937,0.096285,0.437887,0.308524,0.085418,0.496561,0.514715,0.734062,0.505002,0.647657,0.332858,0.650416,0.324124,0.365645,0.593709,0.491736,0.434947,0.091356,0.090383,0.220210,0.374265,0.196220,0.093125,0.489566,0.529580,0.680280
2,0.813604,1.000000,0.000000,0.000000,0.000000,0.000000,0.804704,0.000000,0.322532,0.000000,0.631874,0.000000,0.316278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.557477,0.513923,0.0,0.548409,0.000000,0.000000,0.680119,0.000000,0.0,0.300220,0.0,0.440762,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.351635,...,0.294509,0.0,0.575488,0.231637,0.000000,0.516042,0.000000,0.357674,0.789883,0.627887,0.000000,0.262891,0.000000,0.349878,0.357591,0.000000,0.411962,0.000000,0.000000,0.610322,0.632637,0.814286,0.289707,0.481025,0.000000,0.483075,0.000000,0.000000,0.340596,0.514091,0.534593,0.000000,0.000000,0.270660,0.000000,0.241174,0.000000,0.601726,0.553656,0.341424
3,0.152600,0.000000,1.000000,0.884671,0.570705,0.000000,0.000000,0.000000,0.000000,0.290645,0.000000,0.000000,0.467949,0.000000,0.691079,0.200940,0.246990,0.000000,0.000000,0.000000,0.000000,0.0,0.267096,0.000000,0.000000,0.000000,0.297578,0.0,0.302712,0.0,0.000000,0.000000,0.304461,0.000000,0.207619,0.419287,0.0,0.000000,0.506727,0.000000,...,0.435742,0.0,0.166688,0.165543,0.423647,0.000000,0.

In [ ]:
# get one-hot-encoded table of movie genres
df_movie_genre = pd.get_dummies(df_movies['genres'])

# get total of users id in dataframe
n_users = len(df_user_items.index)

# get user-genre similarities dataframe
df_user_genre = pd.DataFrame(columns = recommender_df.columns)
for i in range(1,n_users+1):
    df_user_genre.loc[i] = df_movie_genre.mul(df_user_items.loc[i], axis=0).mean() # multiply one-hot-encoded movie column with user-items dataframe from recommender 1
df_user_genre

genres,Adventure|Animation|Children|Comedy|Fantasy,Adventure|Children|Fantasy,Comedy|Romance,Comedy|Drama|Romance,Comedy,Action|Crime|Thriller,Adventure|Children,Action,Action|Adventure|Thriller,Comedy|Horror,Adventure|Animation|Children,Drama,Action|Adventure|Romance,Crime|Drama,Drama|Romance,Action|Comedy|Crime|Drama|Thriller,Comedy|Crime|Thriller,Crime|Drama|Horror|Mystery|Thriller,Drama|Sci-Fi,Children|Drama,Adventure|Drama|Fantasy|Mystery|Sci-Fi,Mystery|Sci-Fi|Thriller,Children|Comedy,Drama|War,Action|Crime|Drama,Action|Adventure|Fantasy,Comedy|Drama|Thriller,Mystery|Thriller,Animation|Children|Drama|Musical|Romance,Crime|Mystery|Thriller,Adventure|Drama,Drama|Thriller,Comedy|Crime,Action|Sci-Fi|Thriller,Action|Comedy|Horror|Thriller,Comedy|Drama,Documentary,Action|Crime|Drama|Thriller,Crime|Drama|Romance,Action|Adventure|Drama,...,Adventure|Romance|Sci-Fi,Horror|Sci-Fi|Western,Action|Adventure|Children|Comedy|Sci-Fi,Action|Adventure|Animation|Comedy|Sci-Fi,Horror|Romance|Sci-Fi,Action|Adventure|Children|Mystery|Sci-Fi,Comedy|Crime|Sci-Fi,Action|Comedy|Fantasy|Sci-Fi,Adventure|Animation|Children|Comedy|Drama|Fantasy,Adventure|Children|Comedy|Sci-Fi,Action|Animation|Crime|Sci-Fi,Action|Adventure|Romance|Sci-Fi,Action|Crime|Drama|Sci-Fi,Animation|Children|Comedy|Drama|Romance,Action|Fantasy|Mystery,Comedy|Horror|Mystery,Comedy|Crime|Fantasy,Animation|Horror|Mystery,Comedy|Romance|Western,Adventure|Drama|Fantasy|Sci-Fi,Adventure|Children|Drama|Sci-Fi,Adventure|Children|Comedy|Drama|Fantasy|Sci-Fi,Animation|Fantasy|Horror|Mystery,Adventure|Animation|Fantasy|Horror|Sci-Fi,Action|Animation|Crime|Drama,Action|Adventure|Animation|Drama|Fantasy|Sci-Fi,Action|Animation|Mystery,Animation|Drama|Sci-Fi,Animation|Drama|Fantasy|Romance,Action|Adventure|Comedy|Fantasy|Sci-Fi|Thriller,Action|Adventure|Fantasy|Horror|Thriller,Comedy|Sci-Fi|War,Comedy|Mystery|Romance|Thriller,Fantasy|Horror|Sci-Fi|Western,Animation|Crime|Drama,Adventure|Mystery|Sci-Fi|Thriller,Action|Comedy|Crime|Horror,Action|Adventure|Children|Sci-Fi,Action|Adventure|Comedy|Fantasy|Sci-Fi,Action|Animation|Comedy|Fantasy
1,0.002034,0.004694,0.089590,0.071089,0.226125,0.014416,0.007837,0.015100,0.010423,0.014395,0.004472,0.285284,0.002539,0.035824,0.100540,0.001843,0.002617,0.000428,0.008005,0.004853,0.000533,0.002136,0.021506,0.031581,0.011604,0.008605,0.001989,0.008325,0.000967,0.003783,0.011828,0.046612,0.021611,0.011471,0.001363,0.106071,0.063544,0.015420,0.004685,0.005630,...,0.0,0.000642,0.0,0.0,0.0,0.000366,0.0,0.0,0.0,0.000410,0.000537,0.0,0.000367,0.000532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001380,0.003362,0.063143,0.049247,0.158062,0.010152,0.005397,0.010670,0.007390,0.010150,0.002943,0.199783,0.001768,0.025119,0.070818,0.001291,0.001883,0.000303,0.005687,0.003308,0.000373,0.001537,0.014862,0.022355,0.008196,0.005914,0.001504,0.005876,0.000644,0.002625,0.008189,0.032781,0.015045,0.008221,0.000928,0.074467,0.044801,0.010616,0.003329,0.004058,...,0.0,0.000399,0.0,0.0,0.0,0.000222,0.0,0.0,0.0,0.000339,0.000338,0.0,0.000278,0.000362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000995,0.002367,0.045359,0.035701,0.113950,0.007298,0.003832,0.007890,0.005307,0.007514,0.002096,0.143610,0.001217,0.018107,0.051152,0.000988,0.001384,0.000198,0.004014,0.002366,0.000298,0.001085,0.010644,0.016050,0.005878,0.004207,0.001098,0.004244,0.000471,0.001864,0.005875,0.023619,0.010892,0.005936,0.000700,0.054157,0.032420,0.007646,0.002286,0.002962,...,0.0,0.000279,0.0,0.0,0.0,0.000191,0.0,0.0,0.0,0.000223,0.000246,0.0,0.000226,0.000287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.001648,0.003783,0.072583,0.057703,0.183676,0.011639,0.006245,0.012439,0.008484,0.011856,0.003504,0.231791,0.002004,0.029309,0.081690,0.001525,0.002180,0.000337,0.006419,0.003874,0.000454,0.001739,0.017237,0.025662,0.

In [ ]:
# initialize user-movie similarity dataframe
df_user_movie_sim_1 = pd.DataFrame(columns = df_user_items.columns)

# get user-movie similarity 
for i in range(1,n_users+1):
  df_user_movie_sim_1.loc[i] = recommender_df.mul(df_user_genre.loc[i]).sum(axis=1)
df_user_movie_sim_1

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,0.366588,0.220308,0.567280,0.815695,0.564695,0.411073,0.567280,0.193593,0.250303,0.400723,0.815695,0.441255,0.195613,0.673164,0.414427,0.506066,0.614686,0.564695,0.564695,0.778477,0.539538,0.523156,0.411073,0.453649,0.614686,0.673164,0.390724,0.614686,0.423965,0.506066,0.673164,0.282855,0.390724,0.506066,0.357734,0.567280,0.673164,0.339926,0.555279,0.673164,...,0.193575,0.535181,0.564695,0.279702,0.324700,0.594799,0.279742,0.484305,0.326255,0.081292,0.379611,0.482243,0.506066,0.567280,0.564695,0.400206,0.871553,0.081292,0.400723,0.411073,0.273347,0.714596,0.289746,0.614686,0.564695,0.673164,0.673164,0.070410,0.081292,0.501867,0.421131,0.353948,0.871553,0.070410,0.081292,0.394677,0.317299,0.673164,0.202224,0.564695
2,0.255980,0.153657,0.397660,0.571913,0.395029,0.288511,0.397660,0.135261,0.175215,0.280963,0.571913,0.308901,0.136600,0.472131,0.290488,0.354977,0.431504,0.395029,0.395029,0.545808,0.378383,0.366953,0.288511,0.317863,0.431504,0.472131,0.273741,0.431504,0.296439,0.354977,0.472131,0.198229,0.273741,0.354977,0.250128,0.397660,0.472131,0.238408,0.389283,0.472131,...,0.135815,0.374478,0.395029,0.195322,0.227097,0.416909,0.195519,0.338802,0.228113,0.057476,0.265133,0.337022,0.354977,0.397660,0.395029,0.279524,0.610520,0.057476,0.280963,0.288511,0.191229,0.500735,0.203351,0.431504,0.395029,0.472131,0.472131,0.049078,0.057476,0.351036,0.294496,0.247970,0.610520,0.049078,0.057476,0.275722,0.221494,0.472131,0.141388,0.395029
3,0.184786,0.110853,0.287023,0.412812,0.285293,0.207815,0.287023,0.097504,0.126469,0.202459,0.412812,0.223402,0.098525,0.340811,0.209500,0.255964,0.311372,0.285293,0.285293,0.393650,0.272721,0.264664,0.207815,0.228959,0.311372,0.340811,0.197575,0.311372,0.213590,0.255964,0.340811,0.142445,0.197575,0.255964,0.180595,0.287023,0.340811,0.172179,0.280777,0.340811,...,0.097717,0.270392,0.285293,0.140701,0.163469,0.300365,0.141059,0.244222,0.164270,0.041572,0.191134,0.242996,0.255964,0.287023,0.285293,0.201775,0.440809,0.041572,0.202459,0.207815,0.137597,0.361281,0.146079,0.311372,0.285293,0.340811,0.340811,0.035468,0.041572,0.253013,0.212341,0.179036,0.440809,0.035468,0.041572,0.199138,0.160011,0.340811,0.102089,0.285293
4,0.297211,0.178246,0.460973,0.663075,0.458873,0.333706,0.460973,0.156537,0.202994,0.324888,0.663075,0.358993,0.158289,0.547525,0.336108,0.411430,0.499773,0.458873,0.458873,0.632466,0.438309,0.425197,0.333706,0.368485,0.499773,0.547525,0.317349,0.499773,0.343795,0.411430,0.547525,0.229268,0.317349,0.411430,0.290323,0.460973,0.547525,0.276423,0.451140,0.547525,...,0.157233,0.434564,0.458873,0.226658,0.262947,0.482658,0.226547,0.392998,0.264133,0.066463,0.308014,0.391019,0.411430,0.460973,0.458873,0.324602,0.708573,0.066463,0.324888,0.333706,0.221518,0.580858,0.235016,0.499773,0.458873,0.547525,0.547525,0.057124,0.066463,0.406970,0.341727,0.287758,0.708573,0.057124,0.066463,0.320337,0.257613,0.547525,0.164021,0.458873
5,0.217635,0.130637,0.338586,0.487009,0.335994,0.246392,0.338586,0.115123,0.149030,0.239625,0.487009,0.262272,0.116193,0.402114,0.247431,0.302826,0.367674,0.335994,0.335994,0.465290,0.322911,0.312867,0.246392,0.271526,0.367674,0.402114,0.233027,0.367674,0.252877,0.302826,0.402114,0.169840,0.233027,0.302826,0.212706,0.338586,0.402114,0.202865,0.331827,0.402114,...,0.116243,0.318514,0.335994,0.166623,0.193914,0.355834,0.166306,0.288899,0.194613,0.048920,0.226030,0.287148,0.302826,0.338586,0.335994,0.237718,0.519647,0.048920,0.239625,0.246392,0.163454,0.426692,0.174386,0.367674,0.335994,0.402114,0.402114,0.041658,0.048920,0.299283,0.250999,0.211069,0.519647,0.041658,0.048920,0.234350,0.1

In [ ]:
# get user recommendation 
recommendation(df_user_movie_sim_1, dataset_rating, dataset_movies)

Streaming output truncated to the last 5000 lines.
16 . New York Stories (1989)
17 . How to Kill Your Neighbor's Dog (2000)
18 . Avanti! (1972)
19 . The Longest Week (2014)
20 . Masked & Anonymous (2003)


User id =  403
Recommended movies: 
1 . Strike! (a.k.a. All I Wanna Do, The Hairy Bird) (1998)
2 . Parenthood (1989)
3 . Inkwell, The (1994)
4 . Mystery Train (1989)
5 . Lucky You (2007)
6 . I'll Do Anything (1994)
7 . North Dallas Forty (1979)
8 . Network (1976)
9 . Mr. Saturday Night (1992)
10 . Ro.Go.Pa.G. (1963)
11 . Full Monty, The (1997)
12 . Reefer Madness (a.k.a. Tell Your Children) (1938)
13 . Indian Summer (a.k.a. Alive & Kicking) (1996)
14 . Empire Records (1995)
15 . House of Yes, The (1997)
16 . Funny Bones (1995)
17 . Goon (2011)
18 . With Honors (1994)
19 . Jeff, Who Lives at Home (2012)
20 . Kitchen Stories (Salmer fra kjøkkenet) (2003)


User id =  404
Recommended movies: 
1 . Squid and the Whale, The (2005)
2 . Calendar Girl (1993)
3 . Take Me Home Tonight (2011)
4 

# **Content-Based Filtering (Clustering over tags)**

In [ ]:
# get ratings dataset
df_tags = pd.read_csv(drive+'tags.csv', sep=',')
df_tags.head(10)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [ ]:
# lowercase folding tag column values
df_tags['tag'] = df_tags['tag'].str.lower()

In [ ]:
# get unique values of tag column
x = df_tags['tag'].str.split("|").to_list()
flatten_list = list(chain.from_iterable(x)) 
set(flatten_list)

{'french',
 'scotland',
 'simon pegg',
 'adultery',
 'mafia',
 'beautiful scenery',
 'investor corruption',
 'metaphorical',
 'multiple stories',
 'sisterhood',
 'live action/animation',
 'poor dialogue',
 'philosophical issues',
 'space travel',
 'stand-up comedy',
 'non-linear timeline',
 'trippy',
 'invisibility',
 'ocean',
 'new composer',
 'biopic',
 'witty',
 'tilda swinton',
 'nightclub',
 'depressing',
 'thrilling',
 'prison',
 'lord of the rings',
 'american indians',
 'great humor',
 'whimsical',
 'financial crisis',
 'engrossing adventure',
 'meaningless violence',
 'awkward',
 'ancient rome',
 'trey parker',
 'statue',
 'teachers',
 'philosophy',
 'space epic',
 'may-december romance',
 'cult',
 'psychopaths',
 'british gangster',
 'tolkein',
 'moon',
 'backwards. memory',
 'leonardo dicaprio',
 'race',
 'deep throat',
 'hannibal lecter',
 'long takes',
 'amazing dialogues',
 'monologue',
 'dreamlike',
 'boxing story',
 'ireland',
 'eccentric',
 'nightmare',
 'quentin taran

In [ ]:
# sort tag values by movie id
df_tags = df_tags.sort_values(by=['movieId'])
df_tags

,userId,movieId,tag,timestamp
2886,567,1,fun,1525286013
981,474,1,pixar,1137206825
629,336,1,pixar,1139045764
35,62,2,robin williams,1528843907
34,62,2,magic board game,1528843932
...,...,...,...,...
402,62,187595,star wars,1528934552
528,184,193565,comedy,1537098587
527,184,193565,anime,1537098582
530,184,193565,remaster,1537098592


In [ ]:
# get movie dataset
df_movies = dataset_movies

In [ ]:
# add tag column in movie dataset and set it with empty values
df_movies["tag"] = np.nan
df_movies

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN
9739,193585,Flint (2017),Drama,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN


In [ ]:
# set value of tag in movie dataset
for index,row in df_tags.iterrows():
  idx = df_movies.index[df_movies["movieId"] == row[1]].tolist()[0]
  if pd.isnull(df_movies["tag"].iloc[idx]):
    df_movies["tag"].iloc[idx] = row[2]
  else:
    if row[2] not in df_movies["tag"].iloc[idx]:
      df_movies["tag"].iloc[idx] += "|"+row[2]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
# delete movie rows where tag column value is empty
df_movies = df_movies[~df_movies["tag"].isnull()].iloc[:,[0,1,3]]
df_movies

,movieId,title,tag
0,1,Toy Story (1995),fun|pixar
1,2,Jumanji (1995),robin williams|magic board game|fantasy
2,3,Grumpier Old Men (1995),old|moldy
4,5,Father of the Bride Part II (1995),remake|pregnancy
6,7,Sabrina (1995),remake
...,...,...,...
9681,183611,Game Night (2018),comedy|rachel mcadams|funny
9692,184471,Tomb Raider (2018),adventure|alicia vikander|video game adaptation
9709,187593,Deadpool 2 (2018),josh brolin|ryan reynolds|sarcasm
9710,187595,Solo: A Star Wars Story (2018),emilia clarke|star wars


In [ ]:
# get dataframe of cosine similarity
recommender_df = count_cosine_similarity(df_movies.tag, df_movies.movieId)
recommender_df

tag,fun|pixar,robin williams|magic board game|fantasy,old|moldy,remake|pregnancy,remake,president|politics,politics|president,mafia,jane austen,hollywood,serial killer,alcoholism,shakespeare,jane austen|in netflix queue,kidnapping,teacher|high school,brad pitt|bruce willis|mindfuck|twist ending|time travel|post apocalyptic|remake|post-apocalyptic,animal movie|pigs|villain nonexistent or not needed for good story,death penalty|nun,twins,emma|jane austen|seen more than once|quotable|paul rudd|funny|chick flick,south africa|in netflix queue,england,journalism,wedding,serial killer|mystery|twist ending,mindfuck|thriller|suspense|twist ending|heist|tricky,adoption|prostitution,writing,music,jekyll and hyde,theater,crime|quirky|off-beat comedy,adam sandler|test tag|golf,muppets,beautiful scenery|oscar (best cinematography)|mel gibson|scotland|sword fight|revenge|medieval|inspirational|historical|epic,assassination,holocaust,dating,mark wahlberg|heroin|addiction|leonardo dicaprio,...,dystopia|jennifer lawrence|love story|revolution|rebellion|politics|nonsense,ethics|ryan reynolds|immortality|ben kingsley,poorly paced|harley quinn|superhero|visually appealing|will smith|poor story|margot robbie|harley quinn's ass|joker|jared leto|horrible directing|bad story|bad writing|batman|good soundtrack|dc comics|comic book|ben affleck|lack of plot,superman|wonderwoman|gal gadot|superhero|action|batman|ben affleck|dark|dc comics,stylish|slick,visually striking|visually appealing|cinematography|tom hardy|survival|leonardo dicarpio,atmospheric|tension,touching|moving,moving,tragic|thought-provoking|philosophical|contemplative,sustainability|dumpster diving,funny|interesting|witty,existential|surreal,suspense|creepy,procedural|murder|human rights|crime|cambodia,nightmare,thought-provoking|suspense,80's,sweet|understated|quirky,crude humor|mindfuck|sarcasm|satire|vulgar,creative|viggo mortensen|individualism|good writing|freedom|building a family,intense,beautiful visuals|visually appealing|thought-provoking|smart|good cinematography|cerebral,visually appealing|bittersweet,heartwarming|funny,action|heroic bloodshed|keanu reeves|secret society|organized crime|hitman|gun tactics|dark hero,predictible plot|heartbreaking|gritty|emotional|dark,paranoia|suspenseful,dystopia|thought provoking|jon hamm|interesting|future,inspiring|tense|brilliant,dark|atmospheric|beautiful|cinematography|philosophical|moody|existentialism|dreamlike,unsettling|uncomfortable|allegorical,dwayne johnson|comedy|funny|action,dreamlike|atmospheric,bad music,comedy|rachel mcadams|funny,adventure|alicia vikander|video game adaptation,josh brolin|ryan reynolds|sarcasm,emilia clarke|star wars,comedy|anime|remaster|gintama
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.178864,0.0,0.0,0.000000
3,0.0,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
5,0.0,0.00

In [ ]:
# get total users from user_movie_matrix from recommender 2
n_users = len(df_user_movie_sim_1.index)

In [ ]:
# get one-hot-encoded movie tag
df_movie_tag = pd.get_dummies(df_movies['tag'])

# get user similarity values to all tag
df_user_tag = pd.DataFrame(columns = recommender_df.columns)
for i in range(1,n_users+1):
    df_user_tag.loc[i] = df_movie_tag.mul(df_user_movie_sim_1.loc[i], axis=0).mean() # multiply one-hot-encoded movie tag with user_movie_matrix from recommender 2
df_user_tag

tag,fun|pixar,robin williams|magic board game|fantasy,old|moldy,remake|pregnancy,remake,president|politics,politics|president,mafia,jane austen,hollywood,serial killer,alcoholism,shakespeare,jane austen|in netflix queue,kidnapping,teacher|high school,brad pitt|bruce willis|mindfuck|twist ending|time travel|post apocalyptic|remake|post-apocalyptic,animal movie|pigs|villain nonexistent or not needed for good story,death penalty|nun,twins,emma|jane austen|seen more than once|quotable|paul rudd|funny|chick flick,south africa|in netflix queue,england,journalism,wedding,serial killer|mystery|twist ending,mindfuck|thriller|suspense|twist ending|heist|tricky,adoption|prostitution,writing,music,jekyll and hyde,theater,crime|quirky|off-beat comedy,adam sandler|test tag|golf,muppets,beautiful scenery|oscar (best cinematography)|mel gibson|scotland|sword fight|revenge|medieval|inspirational|historical|epic,assassination,holocaust,dating,mark wahlberg|heroin|addiction|leonardo dicaprio,...,dystopia|jennifer lawrence|love story|revolution|rebellion|politics|nonsense,ethics|ryan reynolds|immortality|ben kingsley,poorly paced|harley quinn|superhero|visually appealing|will smith|poor story|margot robbie|harley quinn's ass|joker|jared leto|horrible directing|bad story|bad writing|batman|good soundtrack|dc comics|comic book|ben affleck|lack of plot,superman|wonderwoman|gal gadot|superhero|action|batman|ben affleck|dark|dc comics,stylish|slick,visually striking|visually appealing|cinematography|tom hardy|survival|leonardo dicarpio,atmospheric|tension,touching|moving,moving,tragic|thought-provoking|philosophical|contemplative,sustainability|dumpster diving,funny|interesting|witty,existential|surreal,suspense|creepy,procedural|murder|human rights|crime|cambodia,nightmare,thought-provoking|suspense,80's,sweet|understated|quirky,crude humor|mindfuck|sarcasm|satire|vulgar,creative|viggo mortensen|individualism|good writing|freedom|building a family,intense,beautiful visuals|visually appealing|thought-provoking|smart|good cinematography|cerebral,visually appealing|bittersweet,heartwarming|funny,action|heroic bloodshed|keanu reeves|secret society|organized crime|hitman|gun tactics|dark hero,predictible plot|heartbreaking|gritty|emotional|dark,paranoia|suspenseful,dystopia|thought provoking|jon hamm|interesting|future,inspiring|tense|brilliant,dark|atmospheric|beautiful|cinematography|philosophical|moody|existentialism|dreamlike,unsettling|uncomfortable|allegorical,dwayne johnson|comedy|funny|action,dreamlike|atmospheric,bad music,comedy|rachel mcadams|funny,adventure|alicia vikander|video game adaptation,josh brolin|ryan reynolds|sarcasm,emilia clarke|star wars,comedy|anime|remaster|gintama
1,0.0,0.000363,0.000218,0.000808,0.000867,0.000545,0.000194,0.003338,0.001169,0.001339,0.001279,0.001635,0.003224,0.000387,0.000609,0.000502,0.000667,0.000280,0.0,0.002358,0.0,0.000502,0.001777,0.002699,0.002106,0.000667,0.000609,0.000239,0.001961,0.000667,0.000577,0.001473,0.000370,0.000547,0.000397,0.000502,0.000859,0.001733,0.001414,0.0,...,0.0,0.000530,0.000667,0.0,0.000356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000254,0.000152,0.000567,0.000607,0.000382,0.000135,0.002340,0.000820,0.000939,0.000897,0.001146,0.002260,0.000271,0.000428,0.000352,0.000468,0.000196,0.0,0.001652,0.0,0.000352,0.001247,0.001892,0.001478,0.000468,0.000428,0.000168,0.001375,0.000468,0.000405,0.001033,0.000260,0.000384,0.000278,0.000352,0.000602,0.001216,0.000991,0.0,...,0.0,0.000371,0.000468,0.0,0.000248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000183,0.000110,0.000409,0.000438,0.000275,0.000098,0.001690,0.000591,0.000677,0.000647,0.000826,0.001630,0.000196,0.000309,0.000254,0.000338,0.000141,0.0,0.001192,0.0,0.000254,0.000899,0.001364,0.001066,0.000338,0.000309,0.000121,0.0009

In [ ]:
# initialize new user-movie similarity dataframe
df_user_movie_sim_2 = pd.DataFrame(columns = df_user_items.columns)

# map user-movie similarity to dataframe
for i in range(1,n_users+1):
  df_user_movie_sim_2.loc[i] = recommender_df.mul(df_user_tag.loc[i]).sum(axis=1)

df_user_movie_sim_2

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,0.000936,0.001099,0.000218,NaN,0.002672,NaN,0.002719,NaN,NaN,NaN,0.002172,NaN,NaN,0.002172,NaN,0.003929,0.001497,NaN,NaN,NaN,0.001931,0.002236,NaN,NaN,0.001635,0.004258,NaN,0.020119,0.001742,NaN,0.003701,0.004951,0.001789,0.001960,0.002358,0.001005,0.020536,0.004258,NaN,0.002341,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000341,0.000396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000656,0.000770,0.000152,NaN,0.001873,NaN,0.001906,NaN,NaN,NaN,0.001522,NaN,NaN,0.001522,NaN,0.002755,0.001050,NaN,NaN,NaN,0.001354,0.001567,NaN,NaN,0.001146,0.002985,NaN,0.014102,0.001222,NaN,0.002593,0.003471,0.001254,0.001374,0.001652,0.000705,0.014394,0.002985,NaN,0.001642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000239,0.000278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000474,0.000556,0.000110,NaN,0.001351,NaN,0.001375,NaN,NaN,NaN,0.001097,NaN,NaN,0.001097,NaN,0.001989,0.000758,NaN,NaN,NaN,0.000977,0.001131,NaN,NaN,0.000826,0.002154,NaN,0.010175,0.000882,NaN,0.001871,0.002505,0.000904,0.000992,0.001192,0.000509,0.010387,0.002154,NaN,0.001185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000172,0.000200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000761,0.000893,0.000177,NaN,0.002171,NaN,0.002209,NaN,NaN,NaN,0.001764,NaN,NaN,0.001764,NaN,0.003194,0.001217,NaN,NaN,NaN,0.001569,0.001817,NaN,NaN,0.001328,0.003460,NaN,0.016349,0.001417,NaN,0.003008,0.004024,0.001453,0.001593,0.001915,0.000817,0.016688,0.003460,NaN,0.001903,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000277,0.000322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.000559,0.000655,0.000129,NaN,0.001595,NaN,0.001624,NaN,NaN,NaN,0.001297,NaN,NaN,0.001297,NaN,0.002346,0.000895,NaN,NaN,NaN,0.001154,0.001336,NaN,NaN,0.000977,0.002543,NaN,0.012013,0.001041,NaN,0.002208,0.002956,0.001069,0.001170,0.001407,0.000600,0.012262,0.002543,NaN,0.001399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000203,0.000237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.001181,0.001386,0.000275,NaN,0.003371,NaN,0.003430,NaN,NaN,NaN,0.002740,NaN,NaN,0.002740,NaN,0.004957,0.001889,NaN,NaN,NaN,0.002436,0.002820,NaN,NaN,0.002062,0.005372,NaN,0.025381,0.002199,NaN,0.004669,0.006247,0.002257,0.002473,0.002974,0.001269,0.025908,0.005372,NaN,0.002954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000430,0.000500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,0.000770,0.000904,0.000179,NaN,0.002199,NaN,0.002238,NaN,NaN,NaN,0.001786,NaN,NaN,0.001786,NaN,0.003235,0.001233,NaN,NaN,NaN,0.001590,0.001840,NaN,NaN,0.001344,0.003504,NaN,0.016553,0.001435,NaN,0.003043,0.004075,0.001472,0.001613,0.001939,0.000828,0.016896,0.003504,NaN,0.001927,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000280,0.000326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,0.000965,0.001132,0.000225,NaN,0.0

In [ ]:
# convert all nan values with 0
df_user_movie_sim_2 = df_user_movie_sim_2.fillna(0)
df_user_movie_sim_2

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,0.000936,0.001099,0.000218,0.0,0.002672,0.0,0.002719,0.0,0.0,0.0,0.002172,0.0,0.0,0.002172,0.0,0.003929,0.001497,0.0,0.0,0.0,0.001931,0.002236,0.0,0.0,0.001635,0.004258,0.0,0.020119,0.001742,0.0,0.003701,0.004951,0.001789,0.001960,0.002358,0.001005,0.020536,0.004258,0.0,0.002341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000341,0.000396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000656,0.000770,0.000152,0.0,0.001873,0.0,0.001906,0.0,0.0,0.0,0.001522,0.0,0.0,0.001522,0.0,0.002755,0.001050,0.0,0.0,0.0,0.001354,0.001567,0.0,0.0,0.001146,0.002985,0.0,0.014102,0.001222,0.0,0.002593,0.003471,0.001254,0.001374,0.001652,0.000705,0.014394,0.002985,0.0,0.001642,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000239,0.000278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000474,0.000556,0.000110,0.0,0.001351,0.0,0.001375,0.0,0.0,0.0,0.001097,0.0,0.0,0.001097,0.0,0.001989,0.000758,0.0,0.0,0.0,0.000977,0.001131,0.0,0.0,0.000826,0.002154,0.0,0.010175,0.000882,0.0,0.001871,0.002505,0.000904,0.000992,0.001192,0.000509,0.010387,0.002154,0.0,0.001185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000172,0.000200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000761,0.000893,0.000177,0.0,0.002171,0.0,0.002209,0.0,0.0,0.0,0.001764,0.0,0.0,0.001764,0.0,0.003194,0.001217,0.0,0.0,0.0,0.001569,0.001817,0.0,0.0,0.001328,0.003460,0.0,0.016349,0.001417,0.0,0.003008,0.004024,0.001453,0.001593,0.001915,0.000817,0.016688,0.003460,0.0,0.001903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000277,0.000322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000559,0.000655,0.000129,0.0,0.001595,0.0,0.001624,0.0,0.0,0.0,0.001297,0.0,0.0,0.001297,0.0,0.002346,0.000895,0.0,0.0,0.0,0.001154,0.001336,0.0,0.0,0.000977,0.002543,0.0,0.012013,0.001041,0.0,0.002208,0.002956,0.001069,0.001170,0.001407,0.000600,0.012262,0.002543,0.0,0.001399,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000203,0.000237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.001181,0.001386,0.000275,0.0,0.003371,0.0,0.003430,0.0,0.0,0.0,0.002740,0.0,0.0,0.002740,0.0,0.004957,0.001889,0.0,0.0,0.0,0.002436,0.002820,0.0,0.0,0.002062,0.005372,0.0,0.025381,0.002199,0.0,0.004669,0.006247,0.002257,0.002473,0.002974,0.001269,0.025908,0.005372,0.0,0.002954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000430,0.000500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.000770,0.000904,0.000179,0.0,0.002199,0.0,0.002238,0.0,0.0,0.0,0.001786,0.0,0.0,0.001786,0.0,0.003235,0.001233,0.0,0.0,0.0,0.001590,0.001840,0.0,0.0,0.001344,0.003504,0.0,0.016553,0.001435,0.0,0.003043,0.004075,0.001472,0.001613,0.001939,0.000828,0.016896,0.003504,0.0,0.001927,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000280,0.000326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.000965,0.001132,0.000225,0.0,0.0

In [ ]:
# get top-20 recommended movies for every user
recommendation(df_user_movie_sim_2, dataset_rating, dataset_movies)

Streaming output truncated to the last 5000 lines.
16 . My Left Foot (1989)
17 . Buena Vista Social Club (1999)
18 . Little Voice (1998)
19 . Frankenstein (1931)
20 . Purple Rose of Cairo, The (1985)


User id =  403
Recommended movies: 
1 . Himalaya (Himalaya - l'enfance d'un chef) (1999)
2 . Battle of Algiers, The (La battaglia di Algeri) (1966)
3 . Letter to Three Wives, A (1949)
4 . Letter, The (1940)
5 . Alive (1993)
6 . My Family (1995)
7 . River's Edge (1986)
8 . Terrorist, The (a.k.a. Malli) (Theeviravaathi) (1998)
9 . Rainmaker, The (1997)
10 . That's Entertainment (1974)
11 . Gentlemen Prefer Blondes (1953)
12 . Faust (1926)
13 . Alice Doesn't Live Here Anymore (1974)
14 . Brassed Off (1996)
15 . Our Hospitality (1923)
16 . Tokyo Godfathers (2003)
17 . Cold Comfort Farm (1995)
18 . Dresser, The (1983)
19 . Lone Star (1996)
20 . Black Robe (1991)


User id =  404
Recommended movies: 
1 . How to Steal a Million (1966)
2 . Limelight (1952)
3 . Pride & Prejudice (2005)
4 . Midnig